# Azure Cognitive Search LangChain Vector Code Sample
This code demonstrates how to use Azure Cognitive Search with OpenAI and the Azure Cognitive Search LangChain Vector Store
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents --pre`.

In [ ]:
! pip install azure-search-documents==11.4.0b6
! pip install openai
! pip install python-dotenv
! pip install azure-identity
! pip install langchain

## Import required libraries and environment variables

In [2]:
# Import required libraries  
import openai
import os  
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch


## Configure OpenAI Settings

In [3]:
# Configure environment variables  
load_dotenv()  
openai.api_type: str = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
model: str = "text-embedding-ada-002"
engine: str = "text-embedding-ada-002"

## Configure Vector Store Settings

In [4]:
vector_store_address: str = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")  
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
index_name: str = "langchain-vector-demo"

## Create embeddings and vector store instances
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [5]:
engine: str = "text-embedding-ada-002"
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, model=model, chunk_size=1, openai_api_base="https://delegenz-openai.openai.azure.com/", openai_api_type="azure" )
index_name: str = "langchain-vector-demo"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

## Insert text and embeddings into vector store

In [6]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../data/state_of_the_union.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

['ZGRiMGUxZDEtNTRjYS00M2EyLWIxNGYtZjQ3MTdkM2NkNDFi',
 'YTYxMGI2YjMtYWQxMS00NGQxLTk5N2EtOGQ1OTM1YWQ0MGUz',
 'OTM0OWU4ZTUtOTg2MS00ZjhmLWE1MzktZGY5YzZjNDdiZWFk',
 'ZjVlYzBiMzQtNzExMi00YzI0LTgxY2MtZDQ5YWYzMWY1NzQx',
 'Y2FlYTc0MGMtNTc2Yi00YzExLWFmZjktYjY2YjdhNmIxNDhl',
 'NjgxYTgzYzYtZGJhMC00MTEwLWExN2QtNDc2NmZmYTQ3NDg0',
 'MzU1NGMyZjAtYTFhMS00MGNmLTk0MWEtNWYwMWUzNmU0ZjFj',
 'MGY3YzIyYjMtMTNiYy00ZWRkLTllZmYtMzIyMGYxYzg0ZmNj',
 'OGZmYTA3NTItNzk2Mi00ZWE0LTliYzctOTBjNjMxMmUxYmJi',
 'YTgyZjg1ZDItMzM1OC00YWYwLWE3YTYtM2NmYTk4YjUwZmZi',
 'M2Q1ZDI2ZjUtN2Y1Mi00MjI4LWE1ZjEtNTdhM2VmZGUzNjA2',
 'ZTUyMjM3MzQtOGZkNC00Y2ExLTg0ZGItYmM0ZTQ2MWYyNjI4',
 'ZmI2MDhiN2ItYjUwMy00YjQ5LWEzMmMtYzJhMzc3MGViZjg2',
 'ZTRhODBjZTctZDBhZC00NjlhLTg1OGUtNjg5ZTg3ZDVmMGUz',
 'MTliOWQ3NjEtZTczYy00MjA2LTg3ZTYtMDVjYjdiMTA5ZWM4',
 'YTAxODhmZWItY2YxYS00MGZlLTkxYjYtNjViMDhhMGNkODFm',
 'ODYyN2RmY2EtNmYzZC00N2VkLWE2NzUtNmZjZmNmOTdiYmRl',
 'OGI2YTYwNmQtNmNhMi00NzdkLTkzMDktNTViMTU0YTkwYzhm',
 'MjkyMjg3OGItZjU3Yi00YzNkLTk3ZWMtMDUyMjAwNTkx

## Perform a vector similarity search

In [7]:
# Perform a similarity search
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Perform a hybrid search

In [9]:
# Perform a hybrid search
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3, 
    search_type="hybrid"
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
